In [3]:
import pandas as pd

# 1. Load the dataset
df = pd.read_csv('employee_attendance.csv')

# 2. Preview and check initial info
print("Initial shape:", df.shape)
print("Missing values:\n", df.isnull().sum())

# 3. Remove duplicate attendance entries (e.g., same Employee_ID + Date)
df.drop_duplicates(subset=['Employee_ID', 'Date'], keep='first', inplace=True)

# 4. Standardize column names
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]

# 5. Convert Date to datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# 6. Convert clock_in and clock_out to datetime.time
df['clock_in'] = pd.to_datetime(df['clock_in'], errors='coerce').dt.time
df['clock_out'] = pd.to_datetime(df['clock_out'], errors='coerce').dt.time

# 7. Fill missing Clock_In/Clock_Out with '00:00:00' or use domain logic
import datetime
df['clock_in'] = df['clock_in'].fillna(datetime.time(0, 0))
df['clock_out'] = df['clock_out'].fillna(datetime.time(0, 0))

# 8. Standardize Department names (title case)
df['department'] = df['department'].str.strip().str.title()

# 9. Standardize Status field (capitalize)
df['status'] = df['status'].str.strip().str.capitalize()

# 10. Create new column: Duration (if both times are valid)
def calculate_duration(row):
    try:
        clock_in = datetime.datetime.combine(row['date'], row['clock_in'])
        clock_out = datetime.datetime.combine(row['date'], row['clock_out'])
        duration = (clock_out - clock_in).seconds / 3600  # in hours
        return duration if duration >= 0 else 0
    except:
        return 0

df['work_hours'] = df.apply(calculate_duration, axis=1)

# 11. Sort by date and employee
df.sort_values(by=['employee_id', 'date'], inplace=True)

# 12. Save the cleaned dataset
df.to_csv('cleaned_employee_attendance.csv', index=False)

print("✅ Attendance data cleaned and saved to 'cleaned_employee_attendance.csv'")


Initial shape: (5, 7)
Missing values:
 Attendance_ID    0
Employee_ID      3
Date             3
clock_In         3
Clock_Out        3
Department       3
Status           4
dtype: int64
✅ Attendance data cleaned and saved to 'cleaned_employee_attendance.csv'


C:\Users\Ayyappan\AppData\Local\Temp\ipykernel_11288\877327541.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['clock_in'] = pd.to_datetime(df['clock_in'], errors='coerce').dt.time
C:\Users\Ayyappan\AppData\Local\Temp\ipykernel_11288\877327541.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['clock_out'] = pd.to_datetime(df['clock_out'], errors='coerce').dt.time
